In [14]:
import pandas as pd

df = pd.read_csv('cleaned.csv')

df

,Upvotes,Date,Title,Body Text,Link
0,10,1449121338,Breaking Through the Safeties,Finally heard back from all of my safeties! O...,https://www.reddit.com/r/collegeresults/commen...
1,9,1449874637,Virginia Tech ED Deferred :(,Demographics: In-state Asian male\n\nIntended ...,https://www.reddit.com/r/collegeresults/commen...
2,6,1449888267,Accepted To Middlebury ED,Middlebury ED (only application)\n\nno financi...,https://www.reddit.com/r/collegeresults/commen...
3,6,1449900824,Accepted to Vanderbilt School of Engineering ED,Schools: Vanderbilt University (Accepted ED)\n...,https://www.reddit.com/r/collegeresults/commen...
4,7,1449936672,Accepted into Duke Pratt ED,I was just accepted this past Thursday to my f...,https://www.reddit.com/r/collegeresults/commen...
...,...,...,...,...,...
5598,1,1720456114,This Sub as issues...,Why is it that a black//Latine person will sha...,https://www.reddit.com/r/collegeresults/commen...
5599,1,1720471080,EU student bags ivies and top schools,**Demographics**\n\n* Gender: Male \n* Race/Et...,https://www.reddit.com/r/collegeresults/commen...
5600,1,1720484984,"Great Supplementals, Got into T20s",**Demographics**\n\n* Gender: Female\n* Race/E...,https://www.reddit.com/r/collegeresults/commen...
5601,1,1720485521,"Psych Major, Got into T20s, Great Essays",**Demographics**\n\n* Gender: Female\n* Race/E...,https://www.reddit.com/r/collegeresults/commen...


In [15]:
df2 = df
df2.drop(['Upvotes', 'Date', 'Title', 'Link'], axis=1, inplace=True)


df2 = df2.tail(10) #only lsat 10 posts

df2.head()


,Body Text
5593,**Demographics**\n\n* Gender: Male\n* Race/Eth...
5594,"Hey y'all, first time posting just wanted to k..."
5595,**Demographics**\n\n* Gender: Male\n* Race/Eth...
5596,I am an Asian from a top bay area high school....
5597,**Demographics**\n\n* Female \n* Chinese :D \n...


In [19]:
import re


def extractData(row, pattern):
    text = str(row['Body Text'])
    pattern = re.compile(pattern, re.IGNORECASE)
    match = pattern.search(text)
    if match:
        return match.group(0)
    else:
        return -1
    


def extractGender(row):
    text = str(row['Body Text'])
    pattern = re.compile(r"Gender:.*", re.IGNORECASE)
    match = pattern.search(text)
    if match:
        return match.group(0)
    else:
        return -1
    

def extractRace(row):
    text = str(row['Body Text'])
    pattern = re.compile(r"Race\/Ethnicity:.*", re.IGNORECASE)
    match = pattern.search(text)
    if match:
        return match.group(0)
    else:
        return -1
    

def extractResidence(row):
    text = str(row['Body Text'])
    pattern = re.compile(r"Residence:.*", re.IGNORECASE)
    match = pattern.search(text)
    if match:
        return match.group(0)
    else:
        return -1

def extractIncome(row):
    text = str(row['Body Text'])
    pattern = re.compile(r"Income Bracket:.*", re.IGNORECASE)
    match = pattern.search(text)
    if match:
        return match.group(0)
    else:
        return -1
    
def extractSchoolType(row):
    text = str(row['Body Text'])
    pattern = re.compile(r"Type of School:.*", re.IGNORECASE)
    match = pattern.search(text)
    if match:
        return match.group(0)
    else:
        return -1
    
def extractHooks(row):
    text = str(row['Body Text'])
    pattern = re.compile(r"(Hooks \(Recruited Athlete, URM, First-Gen, Geographic, Legacy, etc.\):.*)|(Hooks:.*)", re.IGNORECASE)
    match = pattern.search(text)
    if match:
        return match.group(0)
    else:
        return -1
    


new_df = pd.DataFrame()
new_df['Body Text'] = df2['Body Text']
new_df['Gender'] = df2.apply(extractData, re.compile(), axis=1)

new_df['Gender'] = df2.apply(extractGender, axis=1)
new_df['Race'] = df2.apply(extractRace, axis=1)
new_df['Residence'] = df2.apply(extractResidence, axis=1)
new_df['Income'] = df2.apply(extractIncome, axis=1)
new_df['School Type'] = df2.apply(extractSchoolType, axis=1)
new_df['Hooks'] = df2.apply(extractHooks, axis=1)


new_df

,Body Text,Gender,Race,Residence,Income,School Type,Hooks
5593,**Demographics**\n\n* Gender: Male\n* Race/Eth...,Gender: Male,Race/Ethnicity: Black (Togolese/Nigerian),Residence: MA,Income Bracket: 80K-100K,Type of School: non-competitive small private ...,"Hooks: urm, lgbtq+, fgli"
5594,"Hey y'all, first time posting just wanted to k...",Gender: Male,Race/Ethnicity: Mexican,Residence: WI,Income Bracket: < 70K for family of 7 includin...,Type of School: Fairly Competitive Public Scho...,"Hooks: First-gen, low-income"
5595,**Demographics**\n\n* Gender: Male\n* Race/Eth...,Gender: Male,Race/Ethnicity: East Asian,Residence: FL,Income Bracket: Upper Class,Type of School: competitive public,Hooks: None
5596,I am an Asian from a top bay area high school....,-1,-1,-1,-1,-1,-1
5597,**Demographics**\n\n* Female \n* Chinese :D \n...,-1,-1,-1,-1,-1,-1
5598,Why is it that a black//Latine person will sha...,-1,-1,-1,-1,-1,-1
5599,**Demographics**\n\n* Gender: Male \n* Race/Et...,Gender: Male,Race/Ethnicity: White,Residence: European Union,Income Bracket: \~$120k,-1,"Hooks (Recruited Athlete, URM, First-Gen, Geog..."
5600,**Demographics**\n\n* Gender: Female\n* Race/E...,Gender: Female,Race/Ethnicity: Black (African American),Residence: IL,Income Bracket: 50k-75k,Type of School: Private Catholic,"Hooks (Recruited Athlete, URM, First-Gen, Geog..."
5601,**Demographics**\n\n* Gender: Female\n* Race/E...,Gender: Female,Race/Ethnicity: Black (African American),Residence: IL,Income Bracket: 50k-75k,Type of School: Private Catholic,"Hooks (Recruited Athlete, URM, First-Gen, Geog..."
5602,People always seem to mention race but never g...,-1,-1,-1,-1,-1,-1
